In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""

"""

# Import the autoreload extension
%load_ext autoreload

# Set autoreload to reload all modules (except those excluded by %aimport)
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" #for GPU inference
from glob import glob
import pickle5 as pickle
import pretty_midi as pm
from gen_utils import bass_trans_ev_model_tf, generate_bass_ev_trans_tf, create_enc, create_onehot_enc



In [ ]:
thr_measures = 16
thr_max_tokens = 800
thr_min_tokens = 50
dec_seq_length = 793

In [3]:
'''load Encoders pickle for onehotencoders'''

#encoders pickle is created during pre-processing
encoders_trans = './aux_files/bass_encoders_cp.pickle'

    
with open(encoders_trans, 'rb') as handle:
    TransEncoders = pickle.load(handle)
#[Encoder_RG, Decoder_Bass]

In [4]:
'''Load Inference Transformer. You may download pre-trained model based 
on the paper. See instructions in ReadME.md'''
trans_bass_hb = bass_trans_ev_model_tf(TransEncoders) 


'''Set Temperature'''
temperature = 0.9

'''Load MIDI files with Guitar (1st) and Bass (2nd). See examples in midi_in folder'''
'''max 16 bars'''
#input folder (put txt token files of rg only here)
inp_path = glob('./tokens_in/*.txt')
#output folder
out_path = './tokens_out/'


Loading Hybrid Music Transformer
Latest checkpoint restored!


In [16]:
for trk in inp_path:
    #get name
    trk_name = trk.split('\\')[-1][:-4] #you may change it depending your OS
    print('Generating..', trk_name)
    
    # PREPROCESSING (get the sequence of tokens)
    rg_sequence = []
    
    with open(trk, 'r') as rg_file:
        # Retrieve the sequence of tokens of the 16 first bars
        rg_lines = rg_file.readlines()
        count_measures = 0
        rg_token_count = 0
        for line in rg_lines:
            rg_token_count+=1
            if line.strip() == "new_measure":
                count_measures+=1
                if count_measures == thr_measures:
                    break
        
            rg_sequence.append(line.strip())
    
    # POST PROCESSING
    #create the Encoder: convert tokens to one-hot encoding
    Enc_Input = create_onehot_enc(rg_sequence, TransEncoders)
    #padding (add 0s to the input until it reaches length 793)
    Enc_Input = Enc_Input + [0]*(dec_seq_length-len(Enc_Input))
        
    # call generation functions
    bass_HB = generate_bass_ev_trans_tf(trans_bass_hb, TransEncoders, temperature, Enc_Input, dec_seq_length=dec_seq_length)      
    # save token files to be passed to the tokens2gp5 algorithm
    save_path = out_path+trk_name
    save_path.replace('_rythmic', '_with_bass.txt')
    bass_HB.write(save_path)
    

Generating.. Metallica - Enter Sandman (5)_rythmic


ValueError: Exception encountered when calling layer "relative_global_attention_3" "                 f"(type RelativeGlobalAttention).

Received incompatible tensor with shape (793, 24) when attempting to restore variable with shape (545, 24) and name model/decoder/dec_layers/1/attn2/relative_embeddings/.ATTRIBUTES/VARIABLE_VALUE.

Call arguments received by layer "relative_global_attention_3" "                 f"(type RelativeGlobalAttention):
  • v=tf.Tensor(shape=(1, 263, 1024), dtype=float32)
  • k=tf.Tensor(shape=(1, 263, 1024), dtype=float32)
  • q=tf.Tensor(shape=(1, 1, 192), dtype=float32)
  • mask=tf.Tensor(shape=(1, 1, 1, 263), dtype=float32)

In [15]:
len(Enc_Input)

263

In [13]:
dec_seq_length

261